In [54]:
from collections import defaultdict
import copy
import json

from json_extractor import from_file_get_n_docs
from tagged_document_parsing_lib import all_possible_mentions_for_surface_forms

from stanford_parser import parse_stanford_doc
from wikifier_parser import parse_wikifier_doc

# Read in Data

In [55]:
stanford_file_name = './50_tagged_by_stanford.jsonl'
wikifier_file_name = './100_tagged_by_wikifier.jsonl'
tagme_file_name = './100_tagged_by_tagme_longtext_0_epsilon_dot1_includecategories_includeallspots.jsonl'

In [56]:
stanford_docs = from_file_get_n_docs(stanford_file_name, 10)

In [57]:
wikifier_docs = from_file_get_n_docs(wikifier_file_name, 10)

In [58]:
tagme_docs = from_file_get_n_docs(tagme_file_name, 10)

# Parse Stanford
From here on, parsing means, converting the output from the stanford tagger into one dictionary with two main keys: entities and tagged-words.

Entities is a map with the entity surface_form and type in json form as a key, and a list of mentions in the doc as value.

tagged-words is a map with a token word index as key, and the mention in which surface_form it is contained as value.

In [59]:
stanford_parsed_docs = [parse_stanford_doc(doc) for doc in stanford_docs]

# Parse Wikifier

In [73]:
wikifier_parsed_docs = [parse_wikifier_doc(doc) for doc in wikifier_docs]

# Parse Tagme

In [ ]:
def parse_tagme_doc(doc):
    indexed_tagme_annotations = defaultdict(lambda: [])
    for annotation in doc['annotations']:
        all_possible_annotation_indexes = all_types_for_surface_form(annotation['spot'])
        for index in all_possible_annotation_indexes:
            indexed_tagme_annotations[json.dumps(index, ensure_ascii=False)].append(annotation)
    return indexed_tagme_annotations

In [80]:
tokens = []
map(tokens.extend, [a['spot'].split() for a in tagme_docs[0]['annotations']])
len(tokens), tokens

(137,
 [u'VETERANS',
  u'saluted',
  u'Worcester',
  u'breakfast',
  u'club',
  u'won',
  u'hearts,',
  u'minds',
  u'HM',
  u'Forces',
  u'met',
  u'Postal',
  u'Order',
  u'Foregate',
  u'Street',
  u'10am',
  u'socialise',
  u'eat',
  u'hunger',
  u'loneliness',
  u'marching',
  u'orders',
  u'Dave',
  u'Carney',
  u'aged',
  u'Hill',
  u'set',
  u'up',
  u'clubs',
  u'country',
  u'you',
  u'can',
  u'picture',
  u'we',
  u'good',
  u'Five',
  u'out',
  u'saw',
  u'article',
  u'newspaper',
  u'turned',
  u'old',
  u'chap',
  u'travel',
  u'Droitwich',
  u'parade',
  u'hours',
  u'generated',
  u'lot',
  u'interest',
  u'estimate',
  u'who',
  u'next',
  u'month',
  u'meeting',
  u'will',
  u'people',
  u'Onwards',
  u'and',
  u'upwards',
  u'management',
  u'pub',
  u'hospitable',
  u'bent',
  u'backwards',
  u'us',
  u'looked',
  u'after',
  u'well',
  u'the',
  u'best',
  u'best',
  u'choice',
  u'choice',
  u'of',
  u'venue',
  u'reserved',
  u'armed',
  u'forces',
  u'Promoted

In [65]:
tagme_docs[0].keys()

[u'lang', u'timestamp', u'api', u'time', u'test', u'annotations']

In [74]:
len(wikifier_parsed_docs[0]['tokens'])

404

In [75]:
len(stanford_parsed_docs[0]['tokens'])

404